In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

## Useful functions
The functions below provide model loading, data loading and preprocessing, Grad-CAM, and inference capabilities.

In [ ]:
def create_session():
    """Helper function for session creation"""
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    return sess

def load_graph(meta_file):
    """Creates new graph and session"""
    graph = tf.Graph()
    with graph.as_default():
        # Create session and load model
        sess = create_session()

        # Load meta file
        print('Loading meta graph from ' + meta_file)
        saver = tf.train.import_meta_graph(meta_file, clear_devices=True)
    return graph, sess, saver

def load_ckpt(ckpt, sess, saver):
    """Helper for loading weights"""
    # Load weights
    if ckpt is not None:
        print('Loading weights from ' + ckpt)
        saver.restore(sess, ckpt)

def load_labels(label_file):
    """Loads image filenames, classes, and bounding boxes"""
    fnames, classes, bboxes = [], [], []
    with open(label_file, 'r') as f:
        for line in f.readlines():
            fname, cls, xmin, ymin, xmax, ymax = line.strip('\n').split()
            fnames.append(fname)
            classes.append(int(cls))
            bboxes.append((int(xmin), int(ymin), int(xmax), int(ymax)))
    return fnames, classes, bboxes

def load_and_preprocess(image_file, bbox=None, width=512, height=512):
    """Loads and preprocesses images for inference"""
    # Load and crop image
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    if bbox is not None:
        image = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]
    image = cv2.resize(image, (width, height), cv2.INTER_CUBIC)

    # Convert to float in range [0, 1] and stack to 3-channel
    image = image.astype(np.float32) / 255.0
    image = np.expand_dims(np.stack((image, image, image), axis=-1), axis=0)
    
    return image


def make_gradcam_graph(graph):
    """Adds additional ops to the given graph for Grad-CAM"""
    with graph.as_default():
        # Get required tensors
        final_conv = graph.get_tensor_by_name(FINAL_CONV_TENSOR)
        logits = graph.get_tensor_by_name(LOGITS_TENSOR)
        preds = graph.get_tensor_by_name(CLASS_PRED_TENSOR)

        # Get gradient
        top_class_logits = logits[0, preds[0]]
        grads = tf.gradients(top_class_logits, final_conv)[0]

        # Comute per-channel average gradient
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        
    return final_conv, pooled_grads
    

def run_gradcam(final_conv, pooled_grads, sess, image):
    """Creates a Grad-CAM heatmap"""
    with graph.as_default():
        # Run model to compute activations, gradients, predictions, and confidences
        final_conv_out, pooled_grads_out, class_pred, class_prob = sess.run(
            [final_conv, pooled_grads, CLASS_PRED_TENSOR, CLASS_PROB_TENSOR],
            feed_dict={IMAGE_INPUT_TENSOR: image, TRAINING_PH_TENSOR: False})
        final_conv_out = final_conv_out[0]
        class_pred = class_pred[0]
        class_prob = class_prob[0, class_pred]
        
        # Compute heatmap as gradient-weighted mean of activations
        for i in range(pooled_grads_out.shape[0]):
            final_conv_out[..., i] *= pooled_grads_out[i]
        heatmap = np.mean(final_conv_out, axis=-1)

        # Convert to [0, 1] range
        heatmap = np.maximum(heatmap, 0)/np.max(heatmap)
        
        # Resize to image dimensions
        heatmap = cv2.resize(heatmap, (image.shape[2], image.shape[1]))
        
    return heatmap, class_pred, class_prob

    
def run_inference(graph, sess, images, batch_size=1):
    """Runs inference on one or more images"""
    # Create feed dict
    feed_dict = {TRAINING_PH_TENSOR: False}

    # Run inference
    with graph.as_default():
        classes, confidences = [], []
        num_batches = int(np.ceil(images.shape[0]/batch_size))
        for i in range(num_batches):
            # Get batch and add it to the feed dict
            feed_dict[IMAGE_INPUT_TENSOR] = images[i*batch_size:(i + 1)*batch_size, ...]

            # Run images through model
            preds, probs = sess.run([CLASS_PRED_TENSOR, CLASS_PROB_TENSOR], feed_dict=feed_dict)

            # Add results to list
            classes.append(preds)
            confidences.append(probs)

    classes = np.concatenate(classes, axis=0)
    confidences = np.concatenate(confidences, axis=0)

    return classes, confidences


def stacked_bar(ax, probs):
    """Creates a stacked bar graph of slice-wise predictions"""
    x = list(range(probs.shape[0]))
    width = 0.8
    ax.bar(x, probs[:, 0], width, color='g')
    ax.bar(x, probs[:, 1], width, bottom=probs[:, 0], color='r')
    ax.bar(x, probs[:, 2], width, bottom=probs[:, :2].sum(axis=1), color='b')
    ax.set_ylabel('Confidence')
    ax.set_xlabel('Slice Index')
    ax.set_title('Class Confidences by Slice')
    ax.legend(CLASS_NAMES, loc='upper right')

## Loading the model and checkpoint
This cell loads the graph and checkpoint, and also adds additional ops for producing Grad-CAM visualizations.

In [ ]:
# Create full paths
meta_file = os.path.join(MODEL_DIR, META_NAME)
ckpt = os.path.join(MODEL_DIR, CKPT_NAME)

# Load metagraph and create session
graph, sess, saver = load_graph(meta_file)

# Load checkpoint
with graph.as_default():
    load_ckpt(ckpt, sess, saver)
final_conv, pooled_grads = make_gradcam_graph(graph)

## Inference with Grad-CAM
The image files may be set to any images you wish to test. Grad-CAM heatmaps will be computed for the predicted class to show which regions are being leveraged by the model during inference.

In [ ]:
# Select image file
idx = 22585
filenames, classes, bboxes = load_labels(LABEL_FILE)
image_file = os.path.join(IMAGE_DIR, filenames[idx])
cls = classes[idx]
bbox = bboxes[idx]

In [ ]:
# Prepare image
image = load_and_preprocess(image_file, bbox)

# Run Grad-CAM
heatmap, class_pred, class_prob = run_gradcam(
    final_conv, pooled_grads, sess, image)

# Show image
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
plt.subplots_adjust(hspace=0.01)
ax[0].imshow(image[0])
plt.suptitle('Predicted Class: {} ({:.3f} confidence)\nTrue Class: {}'.format(CLASS_NAMES[class_pred], class_prob, CLASS_NAMES[cls]))
ax[1].imshow(image[0])
ax[1].imshow(heatmap, cmap='jet', alpha=0.4)

print('**DISCLAIMER**')
print('Do not use this prediction for self-diagnosis. '
      'You should check with your local authorities for '
      'the latest advice on seeking medical assistance.')
plt.show()

In [ ]:
test_df = pd.read_csv("/kaggle/input/train-val-test-datasets/test_df.csv")

In [ ]:
print(image_file)
print(image_file2)
image_file1

In [ ]:
LABEL_FILE

In [ ]:
plt.figure(figsize=(14,10),dpi=100)
plt.subplots_adjust(hspace=0.01)
# 绘制第一个子图
# Prepare image
# Select image file
idx = 19574
filenames, classes, bboxes = load_labels(LABEL_FILE)
image_file = os.path.join(IMAGE_DIR, filenames[idx])
cls = classes[idx]
bbox = bboxes[idx]
image = load_and_preprocess(image_file, bbox)
# Run Grad-CAM
heatmap, class_pred, class_prob = run_gradcam(
    final_conv, pooled_grads, sess, image)
plt.subplot(2,3,1)
plt.axis('off')
plt.imshow(image[0])
plt.subplot(2,3,4)
plt.axis('off')
plt.imshow(image[0])
plt.imshow(heatmap, cmap='jet', alpha=0.4)
plt.tight_layout()

# Select image file
idx1 = 18594
filenames, classes, bboxes = load_labels(LABEL_FILE)
image_file1 = os.path.join(IMAGE_DIR, filenames[idx1])
cls1 = classes[idx1]
bbox1 = bboxes[idx1]
image1 = load_and_preprocess(image_file1, bbox1)
# Run Grad-CAM
heatmap1, class_pred1, class_prob1 = run_gradcam(
    final_conv, pooled_grads, sess, image1)
plt.subplot(2,3,2)
plt.axis('off')
plt.imshow(image1[0])
plt.subplot(2,3,5)
plt.axis('off')
plt.imshow(image1[0])
plt.imshow(heatmap1, cmap='jet', alpha=0.4)
plt.tight_layout()

# Select image file
idx2 = 18895
filenames, classes, bboxes = load_labels(LABEL_FILE)
image_file2 = os.path.join(IMAGE_DIR, filenames[idx2])
cls2 = classes[idx2]
bbox2 = bboxes[idx2]
image2 = load_and_preprocess(image_file2, bbox2)
# Run Grad-CAM
heatmap2, class_pred2, class_prob2 = run_gradcam(
    final_conv, pooled_grads, sess, image2)
plt.subplot(2,3,3)
plt.axis('off')
plt.imshow(image2[0])
plt.subplot(2,3,6)
plt.axis('off')
plt.imshow(image2[0])
plt.imshow(heatmap2, cmap='jet', alpha=0.4)
plt.tight_layout()

plt.savefig("normal.png")